In [10]:
import pickle
import pandas as pd

model2_copy = open("model2.pickle","rb")
model2_copy = pickle.load(model2_copy)

uid_map = open("uid_map.pickle","rb")
uid_map = pickle.load(uid_map)

iid_map = open("iid_map.pickle","rb")
iid_map = pickle.load(iid_map)

train_interactions2 = open("train_interactions2.pickle","rb")
train_interactions2 = pickle.load(train_interactions2)

user_features = open("user_features.pickle","rb")
user_features = pickle.load(user_features)

item_features = open("item_features.pickle","rb")
item_features = pickle.load(item_features)

model2_copy

In [11]:
data = pd.read_csv('../../Post dataset/data.csv')
data.head()

,userID,itemID,rating,item_features,user_features
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,4,artwork|Visual Arts|Graphic Design|Artistic de...,female|undergraduate
1,5df49b32cc709107827fb3c7,5ec1fd0974f7660d73aa0fd5,1,artwork|Drawings,female|undergraduate
2,5df49b32cc709107827fb3c7,5ecd5d417023451e662235c5,4,blog|Computer Technology|Machine Learning,female|undergraduate
3,5df49b32cc709107827fb3c7,5ecb72c0eaff6b0c3a58a48e,2,artwork|Photography|Architecture|Visual Arts|G...,female|undergraduate
4,5df49b32cc709107827fb3c7,5ea5aacd10426255a7aa9b71,3,project|nan,female|undergraduate


In [13]:
def sample_recommendation(
    user_id,
    data,
    uid_map,
    iid_map,
    interactions,
    model,
    num_threads,
    user_features=None,
    item_features=None,
):
    users, items, preds = [], [], []
    item = list(data.itemID.unique())
    for user in user_id:
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)

    all_predictions = pd.DataFrame(data={"userID": users, "itemID": items})
    all_predictions["uid"] = all_predictions.userID.map(uid_map)
    all_predictions["iid"] = all_predictions.itemID.map(iid_map)

    dok_weights = interactions.todok()
    all_predictions["rating"] = all_predictions.apply(
        lambda x: dok_weights[x.uid, x.iid], axis=1
    )

    all_predictions = all_predictions[all_predictions.rating < 1].reset_index(drop=True)
    all_predictions = all_predictions.drop("rating", axis=1)

    all_predictions["prediction"] = all_predictions.apply(
        lambda x: model.predict(
            user_ids=x["uid"],
            item_ids=[x["iid"]],
            user_features=user_features,
            item_features=item_features,
            num_threads=num_threads,
        )[0],
        axis=1,
    )
    
    all_predictions['prediction'] = all_predictions.apply(lambda x: -x['prediction'],axis=1)
    all_predictions.sort_values(by=['prediction'],inplace=True)
    return all_predictions[["userID", "itemID", "prediction"]]

df = sample_recommendation( ['5ec3ba5374f7660d73aa1201'],data, uid_map, iid_map, 
                       interactions=train_interactions2,
                       user_features=user_features,
                       item_features=item_features,
                       model=model2_copy,
                       num_threads=8)

In [15]:
df.head()

,userID,itemID,prediction
79,5ec3ba5374f7660d73aa1201,5ec7aafbec493f4a26558857,112.490555
31,5ec3ba5374f7660d73aa1201,5ec7ad1aec493f4a26558869,112.499001
5,5ec3ba5374f7660d73aa1201,5eb2c11210426255a7aaa052,112.501228
48,5ec3ba5374f7660d73aa1201,5eb1407c10426255a7aa9fdb,112.551956
26,5ec3ba5374f7660d73aa1201,5eca7601ec493f4a26558a62,112.649551
